In [175]:
import pandas as pd
import re
import numpy as np
import pycountry

<hr style='color:white;'>
<p style='color:orange;font-size:20px;text-align:center;'>Data Cleansing</p>
<hr style='color:white;'>

In [176]:
df = pd.read_csv('C:\\csv-data-collect\\job-data.csv',encoding='latin')

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>DUPLICATES</p>
<hr style='color:#BA704F;'>

In [177]:
df.duplicated().sum()

202

In [178]:
df.drop_duplicates(inplace=True)

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Salary</p>
<hr style='color:#BA704F;'>

In [179]:
df['Salary'].replace(np.nan, "0",inplace=True)
df.loc[df['Salary'] == "0"]

def ConvertToIntger(string):
    Money_pattern = re.compile("["
                           u"GBP"
                           u"EUR"
                           u"K"
                           u"+"
                           u"*"
                           "]+"
                           , flags=re.UNICODE)
    return int(Money_pattern.sub(r'', string)) * 1000

df['Salary'] = df['Salary'].apply(ConvertToIntger)

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Company</p>
<hr style='color:#BA704F;'>

In [199]:
df.drop(df.loc[df['Company'].isna()].index,inplace=True)

In [197]:
df.loc[df['Company'].isna()]

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities


<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Experience level</p>
<hr style='color:#BA704F;'>

In [194]:
df['Experience level'].isna().sum()

227

In [195]:
df['Experience level'].fillna('Junior Developer',inplace=True)

<hr style='color:#BA704F;'>
<p style='color:#6C3428;font-size:20px;text-align:center;'>Experience level</p>
<hr style='color:#BA704F;'>

In [203]:
df.loc[df['Job Title'].index == 59]

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities
59,Nielsen,Business Intelligence Developer,"Oldsmar, FL",Full Time,Senior-level,67000,"Agile,APIs,Architecture,Business Intelligence,...","401(k) matching,Health care,Salary bonus,Welln..."


In [202]:
df.loc[df['Experience level'] =='Executive-level']

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities
77,KPMG Australia,Artificial Intelligence Lead | KPMG Futures,"Sydney, Australia",Full Time,Executive-level,65000,"Finance,Research,Testing,,,","Career development,Flex hours,Parental leave,,"
81,Appen,"Director, Artificial Intelligence (AI)","Toronto, Ontario",Full Time,Executive-level,81000,"AWS,Azure,Computer Science,Finance,GANs,GCP","Career development,Startup environment,,,"
104,TTEC Digital,"Director, IVA and AI Solutions - Professional ...",US: Remote,Full Time,Executive-level,73000,"Angular,APIs,Architecture,ASR,AWS,Azure","Career development,,,,"
137,Databricks,"Director, Spark Technical Solutions","Bengaluru, India",Full Time,Executive-level,65000,"AWS,Azure,Big Data,Computer Science,Databricks...","Career development,Startup environment,,,"
148,Publicis Groupe,"Vice President Director, Data Scientist","Philadelphia, PA, United States",Full Time,Executive-level,92000,"AWS,Big Data,Business Analytics,Hadoop,Machine...","Career development,,,,"
...,...,...,...,...,...,...,...,...
3043,Stream Realty,Senior Research Analyst,"Charlotte, NC, United States",Full Time,Executive-level,81000,"Data visualization,Market research,Power BI,Py...","Career development,Team events,,,"
3061,Visa,Senior Director - Head of Internal Data Scienc...,"London, United Kingdom",Full Time,Executive-level,73000,"Business Analytics,Computer Science,Economics,...","Career development,Flex hours,,,"
3120,Visa,"Manager, Business Insights & Data Analytics","Toronto, CANADA, Canada",Full Time,Executive-level,81000,"Computer Science,Data Analytics,Data strategy,...","Flex hours,,,,"
3145,Standard Bank Group,80405862 - Data Engineer,"Johannesburg, South Africa",Full Time,Executive-level,200000,"Agile,APIs,Architecture,Azure,Data Analytics,D...","Career development,Startup environment,,,"
